In [1]:
import pandas as pd

In [2]:
#real excel
xls = pd.ExcelFile('C:/Users/ustjo/Desktop/MM/dolphins/Data/3RSET_CWD_Database.xlsx')

## Prepare Sample Table

In [3]:
effort = xls.parse('Survey Effort')

In [4]:
#remove space in header
effort = effort.rename(columns=lambda x: x.strip())

#remove space in content
df_obj = effort.select_dtypes(['object'])
df_obj
effort[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [5]:
effort.head()

,DATE,AREA,BEAU,KM SEARCHED,SEASON,VESSEL,TYPE
0,2015-12-18,AW,3,5.45,WINTER,32166,3RS ET BASELINE
1,2015-12-18,WL,3,15.50,WINTER,32166,3RS ET BASELINE
2,2015-12-18,WL,4,18.74,WINTER,32166,3RS ET BASELINE
3,2015-12-18,SWL,2,0.29,WINTER,32166,3RS ET BASELINE
4,2015-12-18,SWL,3,3.56,WINTER,32166,3RS ET BASELINE


In [6]:
#filter relevant time for analysis
effort = effort[(effort['DATE']>= '2015-1-1') & (effort['DATE']<= '2016-12-31')]

effort = effort[effort['BEAU'] <= 3]

effort = effort[effort['AREA'] != 'DB' ] 

In [7]:
effort.head()

,DATE,AREA,BEAU,KM SEARCHED,SEASON,VESSEL,TYPE
0,2015-12-18,AW,3,5.45,WINTER,32166,3RS ET BASELINE
1,2015-12-18,WL,3,15.50,WINTER,32166,3RS ET BASELINE
3,2015-12-18,SWL,2,0.29,WINTER,32166,3RS ET BASELINE
4,2015-12-18,SWL,3,3.56,WINTER,32166,3RS ET BASELINE
6,2015-12-21,NEL,1,1.70,WINTER,32166,3RS ET BASELINE


In [8]:
sample_table = {'Sample.Label': [1,2,3,4,5], 'Region.Label':['NWL','SWL','AW','NEL','WL'], 'Effort':[effort['KM SEARCHED'][effort['AREA'] == 'NWL'].sum(), effort['KM SEARCHED'][effort['AREA'] == 'SWL'].sum(),effort['KM SEARCHED'][effort['AREA'] == 'AW'].sum(), effort['KM SEARCHED'][effort['AREA'] == 'NEL'].sum(),effort['KM SEARCHED'][effort['AREA'] == 'WL'].sum()]}

In [9]:
sample_table = pd.DataFrame(sample_table, columns = ['Sample.Label','Region.Label','Effort'])

In [10]:
sample_table.head()

,Sample.Label,Region.Label,Effort
0,1,NWL,1817.059
1,2,SWL,1443.898
2,3,AW,106.830
3,4,NEL,1095.430
4,5,WL,666.389


## Prepare Region Table

In [11]:
region_table = {'Region.Label':['NWL', 'SWL', 'AW', 'NEL', 'WL'], 'Area':[87.415,66.902,2.170,52.190,32.649]}

In [12]:
region_table = pd.DataFrame(region_table, columns = ['Region.Label','Area'])

In [13]:
region_table.head()

,Region.Label,Area
0,NWL,87.415
1,SWL,66.902
2,AW,2.170
3,NEL,52.190
4,WL,32.649


## Prepare obs table

In [14]:
#access sighting sheet
sighting = xls.parse('Sightings')
#remove space in header
sighting = sighting.rename(columns=lambda x: x.strip())
#remove space in content
df_obj = sighting.select_dtypes(['object'])
df_obj
sighting[df_obj.columns] = df_obj.astype("str").apply(lambda x: x.str.strip())
#filter date, CWD, BEAU, On Effort
sighting = sighting[(sighting['DATE'] >= '2015-1-1')& (sighting['DATE'] <= '2016-12-31')]
sighting = sighting[sighting['CWD/FP'] == 'CWD']
sighting = sighting[sighting['BEAU'] <= 3]
sighting = sighting[sighting['EFFORT'] == 'ON']
sighting = sighting[sighting['AREA'] != 'DB' ] 

In [15]:
sighting.head()

,DATE,STG #,TIME,CWD/FP,HRD SZ,AREA,BEAU,PSD,EFFORT,SEEN BY,TYPE,DEC LAT,DEC LON,GRID,SEASON,BOAT ASSOC.,BEHAVIOUR,Remarks,ID INDIVIDUALS,GROUP COMPOSITION
2,2015-12-18,3,1321,CWD,12,SWL,3,498.0,ON,JT,3RS ET BASELINE,22.1791,113.8500,32D,WINTER,NONE,SOCIALIZING / BREACHING,nan,"SLMM001, SLMM002, SLMM003, SLMM004, SLMM005, S...",nan
3,2015-12-18,4,1413,CWD,2,SWL,3,47.0,ON,GC,3RS ET BASELINE,22.1967,113.8696,30F,WINTER,NONE,UNDETERMINED,nan,nan,nan
6,2016-01-04,2,1351,CWD,4,SWL,2,187.0,ON,RO,3RS ET BASELINE,22.2056,113.8877,29H,WINTER,PURSE SEINE,FEEDING,nan,SLMM011,nan
7,2016-01-05,1,1019,CWD,4,NWL,3,242.0,ON,TK,3RS ET BASELINE,22.3908,113.8754,9G,WINTER,NONE,TRAVELING,SCZ,nan,nan
8,2016-01-06,1,924,CWD,6,AW,2,206.0,ON,GC,3RS ET BASELINE,22.3014,113.8786,18G,WINTER,NONE,FEEDING,nan,NLMM001,nan


In [16]:
#sighitng table can be further cut
sighting = sighting[['DATE', 'HRD SZ', 'AREA','PSD']]
sighting.head()

,DATE,HRD SZ,AREA,PSD
2,2015-12-18,12,SWL,498.0
3,2015-12-18,2,SWL,47.0
6,2016-01-04,4,SWL,187.0
7,2016-01-05,4,NWL,242.0
8,2016-01-06,6,AW,206.0


In [17]:
#rename the columns name to distance program format
sighting.columns = ['DATE', 'size', 'Region.Label', 'distance']
sighting.head()

,DATE,size,Region.Label,distance
2,2015-12-18,12,SWL,498.0
3,2015-12-18,2,SWL,47.0
6,2016-01-04,4,SWL,187.0
7,2016-01-05,4,NWL,242.0
8,2016-01-06,6,AW,206.0


## Prepare fiat table

In [18]:
fiat = pd.merge(sighting, sample_table, on=['Region.Label'], how='left')
fiat.head()

,DATE,size,Region.Label,distance,Sample.Label,Effort
0,2015-12-18,12,SWL,498.0,2,1443.898
1,2015-12-18,2,SWL,47.0,2,1443.898
2,2016-01-04,4,SWL,187.0,2,1443.898
3,2016-01-05,4,NWL,242.0,1,1817.059
4,2016-01-06,6,AW,206.0,3,106.830


In [19]:
fiat = pd.merge(fiat, region_table, on=['Region.Label'],how='left')

In [20]:
fiat.head()

,DATE,size,Region.Label,distance,Sample.Label,Effort,Area
0,2015-12-18,12,SWL,498.0,2,1443.898,66.902
1,2015-12-18,2,SWL,47.0,2,1443.898,66.902
2,2016-01-04,4,SWL,187.0,2,1443.898,66.902
3,2016-01-05,4,NWL,242.0,1,1817.059,87.415
4,2016-01-06,6,AW,206.0,3,106.830,2.170


In [21]:
fiat['distance'] = fiat['distance']/1000

In [22]:
fiat.head()

,DATE,size,Region.Label,distance,Sample.Label,Effort,Area
0,2015-12-18,12,SWL,0.498,2,1443.898,66.902
1,2015-12-18,2,SWL,0.047,2,1443.898,66.902
2,2016-01-04,4,SWL,0.187,2,1443.898,66.902
3,2016-01-05,4,NWL,0.242,1,1817.059,87.415
4,2016-01-06,6,AW,0.206,3,106.830,2.170


In [23]:
fiat = fiat.sort_values(by = ['Region.Label'])
fiat.head()

,DATE,size,Region.Label,distance,Sample.Label,Effort,Area
4,2016-01-06,6,AW,0.206,3,106.830,2.170
16,2016-02-18,4,AW,0.149,3,106.830,2.170
114,2016-09-19,2,AW,0.013,3,106.830,2.170
172,2016-12-19,9,NWL,0.119,1,1817.059,87.415
91,2016-07-22,2,NWL,0.065,1,1817.059,87.415


In [24]:
fiat.to_csv('C:/Users/ustjo/Desktop/MM/dolphins/Data/fiat2.txt')
fiat.to_csv('C:/Users/ustjo/Desktop/MM/dolphins/Data/fiat2.csv')